
```
███╗   ███╗██╗███╗   ██╗███████╗ ██████╗ ██████╗ ██╗      █████╗ ██████╗
████╗ ████║██║████╗  ██║██╔════╝██╔════╝██╔═══██╗██║     ██╔══██╗██╔══██╗
██╔████╔██║██║██╔██╗ ██║█████╗  ██║     ██║   ██║██║     ███████║██████╔╝
██║╚██╔╝██║██║██║╚██╗██║██╔══╝  ██║     ██║   ██║██║     ██╔══██║██╔══██╗
██║ ╚═╝ ██║██║██║ ╚████║███████╗╚██████╗╚██████╔╝███████╗██║  ██║██████╔╝
╚═╝     ╚═╝╚═╝╚═╝  ╚═══╝╚══════╝ ╚═════╝ ╚═════╝ ╚══════╝╚═╝  ╚═╝╚═════╝
```
**Run a Minecraft Server on Google Colab!**

---

The script below will run your server. You'll have to create a server first to be able to use it - don't worry, the scripts below will do the majority of the work for you. You might also want to change the default region to your region, check below.

In [18]:
import os
import json
from google.colab import drive
import subprocess
from pyngrok import ngrok

# Update the package lists
!sudo apt update &>/dev/null && echo "apt cache successfully updated" || echo "apt cache update failed, you might receive stale packages"
# Install OpenJDK 17
!sudo apt-get install openjdk-17-jre-headless &>/dev/null && echo "Yay! Openjdk17 has been successfully installed." || echo "Failed to install OpenJdk17."

# Perform Java version check
java_ver = !java -version 2>&1 | awk -F[\"\.] -v OFS=. 'NR==1{print $2}'
if java_ver[0] == "17" :
  print("Openjdk17 is working correctly, you are good to go.")
else:
  print("Openjdk17 doesn't seem to be installed or isn't working, falling back to java", java_ver[0], ". You might experience reduced performance. Minecraft 1.17 and above might fail to launch.")

# Mount Google Drive
drive.mount('/content/drive')
# Change directory to the Minecraft server folder on Google Drive
%cd "/content/drive/My Drive/Minecraft-server"
!ls # list the directory contents (to verify that working directory was changed)

# Import config file.
if os.path.isfile("colabconfig.json"):
  colabconfig = json.load(open("colabconfig.json"))
else:
  colabconfig = {"server_type": "generic"}  # using default if config doesn't exist.
  json.dump(colabconfig, open("colabconfig.json",'w'))

# Server jar names.
jar_list = {'paper': 'server.jar', 'fabric': 'fabric-server-launch.jar', 'generic': 'server.jar', 'forge': 'server.jar'}
jar_name = jar_list[colabconfig["server_type"]]

# Java arguments
if colabconfig["server_type"] == "paper":
  server_flags = "-XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true"
else:
  server_flags = ""  # Aikar's flags might negatively impact performance on non-paper servers.

memory_allocation = "-Xmx6144M -Xms6144M"

# Ngrok Setup
print("Proceeding to use ngrok")

!pip -q install pyngrok

# Ask for the ngrok authtoken
print("Get your authtoken from https://dashboard.ngrok.com/auth")
authtoken = "2UVQqugKuLqdP0nbcj6PfuJFLjy_3ToignpAAneX42y2xnr9p"  # Replace with your ngrok auth token
! ngrok authtoken $authtoken  # login to ngrok

# Sets default ngrok region
ngrok.set_auth_token(authtoken)
url = ngrok.connect(25565, 'tcp')

# Output the ngrok public URL
print('Your server address is ' + url.public_url)

# Start the Minecraft server (using the pre-built forge jar)
print('Starting server...')
!java -Xmx10144M -Xms10144M -jar server.jar nogui

apt cache successfully updated
Yay! Openjdk17 has been successfully installed.
Openjdk17 is working correctly, you are good to go.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content
colabconfig.json  installer.log  run.bat  server.jar
eula.txt	  libraries	 run.sh   user_jvm_args.txt
Proceeding to use ngrok
Get your authtoken from https://dashboard.ngrok.com/auth
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Your server address is tcp://0.tcp.ngrok.io:13149
Starting server...
JVM info: Ubuntu - 17.0.13 - 17.0.13+11-Ubuntu-2ubuntu122.04
java.net.preferIPv4Stack=true
Current Time: 06/01/2025 10:37:16
Host: files.minecraftforge.net [172.67.161.211, 104.21.58.163]
Host: maven.minecraftforge.net [172.67.161.211, 104.21.58.163]
Host: libraries.minecraft.net [13.107.253.40]
Host: launchermeta.mojang.com [13.107.253.40]
Host: piston-meta.mojang.com [13.107.253.40]
Host: sessionserver.mo

# Make-a-Server

The code below will download a server for you and accept the EULA. After running these scripts, your server will be ready to run.

**Download the Minecraft server**

The code below will download Paper, a high-performance fork of the Vanilla server.
Other server platforms can be used by placing the server.jar in the Drive folder manually.

In [21]:
import os
import requests
import json
from google.colab import drive

# Configuration
version = '1.18.2'
forge_version = "43.2.8"  # If you are going to use forge
server_type = 'forge'  # Choose from: fabric, paper, forge

# Mount Google Drive
drive.mount('/content/drive')

# Create the Minecraft server directory if it doesn't exist
server_dir = "/content/drive/My Drive/Minecraft-server"
if not os.path.exists(server_dir):
    os.makedirs(server_dir)

os.chdir(server_dir)

# Fetch the download URL based on server type
if server_type == "paper":
    try:
        a = requests.get(f"https://papermc.io/api/v2/projects/paper/versions/{version}")
        a.raise_for_status()
        builds = a.json()["builds"]

        if not builds:
            raise ValueError(f"No builds available for version {version}")

        latest_build = builds[-1]
        b = requests.get(f"https://papermc.io/api/v2/projects/paper/versions/{version}/builds/{latest_build}")
        b.raise_for_status()

        download_name = b.json()["downloads"]["application"]["name"]
        url = f"https://papermc.io/api/v2/projects/paper/versions/{version}/builds/{latest_build}/downloads/{download_name}"
        jar_name = "server.jar"

    except (requests.RequestException, ValueError) as e:
        print(f"Error fetching PaperMC build: {e}")
        url = None

elif server_type == "fabric":
    url = "https://maven.fabricmc.net/net/fabricmc/fabric-installer/0.7.4/fabric-installer-0.7.4.jar"
    jar_name = "fabric-installer.jar"

elif server_type == "forge":
    url = f"https://maven.minecraftforge.net/net/minecraftforge/forge/1.18.2-40.2.0/forge-1.18.2-40.2.0-installer.jar"
    jar_name = "server.jar"

# Download the file if URL is valid
if url:
    r = requests.get(url)
    if r.status_code == 200:
        jar_path = os.path.join(server_dir, jar_name)
        with open(jar_path, 'wb') as f:
            f.write(r.content)
        print(f'Successfully downloaded {jar_name} to {jar_path}')
    else:
        print(f'Error {r.status_code}! Most likely you entered an unsupported version.')

# Run specific installer if needed
if server_type == "fabric":
    os.system(f"java -jar {jar_name} server -mcversion {version} -downloadMinecraft")
elif server_type == "forge":
    os.system(f"java -jar {jar_name} --installServer --headless")

# Save configuration to a JSON file
colabconfig = {"server_type": server_type}
with open("colabconfig.json", 'w') as config_file:
    json.dump(colabconfig, config_file)

print('Done!')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Successfully downloaded server.jar to /content/drive/My Drive/Minecraft-server/server.jar
Done!


**Automatically accept the EULA**

In [6]:
# Please read the file stored in your server folder before running this command.
# Also, go to https://www.minecraft.net/en-us/eula to read Minecraft's EULA.

# Make sure Drive is mounted
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/My Drive/Minecraft-server"
!echo "eula=true" >> eula.txt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content


# Debug
SSH access to host OS - Thanks to [colab-ssh](https://github.com/WassimBenzarti/colab-ssh).

<p style="color:red;">YOU MIGHT GET BANNED</p>

In [ ]:
#@title Colab-ssh tunnel
#@markdown Execute this cell to open the ssh tunnel. Check [colab-ssh documentation](https://github.com/WassimBenzarti/colab-ssh) for more details.

# Install colab_ssh on google colab
!pip install colab_ssh --upgrade

from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared
ssh_tunnel_password = "<PUT_YOUR_PASSWORD_HERE>" #@param {type: "string"}
launch_ssh_cloudflared(password=ssh_tunnel_password)

In [ ]:
#Get public address (ngrok)
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
## For inspecting the minecraft server directory ##
%cd "/content/drive/My Drive/Minecraft-server"
!ls
